In [1]:
import pandas as pd
import numpy as np
import gurobipy as gp
from gurobipy import GRB

# Imports of Master problem and subproblem
import IRP_LPref
import IRP_MGSP
import Multitours as MT
import utils as ut

# Data

In [2]:
# Read excel file for rows 1 to 9 for demanda data
ncustomers = 8
nvehicles = 4
vehicleCapacity = [60000 for i in range(nvehicles)]
Customers = pd.read_excel('D:/DIEGO/Gent/2nd Year/Thesis/Example2.xlsx', sheet_name='Sheet1', nrows=ncustomers)
Time = pd.read_excel('D:/DIEGO/Gent/2nd Year/Thesis/Example2.xlsx', sheet_name='Sheet1', skiprows=ncustomers+2, usecols="A:C")
Network = Time[["Origin", "Destination"]]
Customers["Tmax"] = Customers["Capacity"] / Customers["Demand"]

### Parameters

In [3]:
# Vehicle cost parameter
psi = [1000 for i in range(nvehicles)]

# Transportation cost
delta = 100

# Delivery cost
# column Phi of df Costumers with indeces 1 to ncustomers
phi = Customers["Phi"].copy()
phi.index = phi.index + 1

# Holding Cost
h = Customers["Holding"].copy()
h.index = h.index + 1

# Demand
d = Customers["Demand"].copy()
d.index = d.index + 1

# Capacity
k = Customers["Capacity"].copy()
k.index = k.index + 1

# Travel time parameter
shape = (ncustomers + 1, ncustomers + 1)
t = np.full(shape, 100)

for index, row in Time.iterrows():
    i = row['Origin']
    j = row['Destination']
    t[i, j] = row['Time']

## Initial set of Multitours and Cycle time

### Random and Basic tours

In [4]:
# Random Multitours
# n_MultiToursR = 10
# MultiToursR = MT.generate_random_multi_tour(n_MultiToursR, Network)
# SingleToursR = [MT.identify_tours(multitour) for multitour in MultiToursR]

# # Basic MultiTours
# MultiToursB = MT.generate_basic_tours2(Network, Time, Customers, ncustomers, vehicleCapacity, delta, phi, h, d)
# SingleToursB = [MT.identify_tours(multitour) for multitour in MultiToursB]
# n_MultiToursB = len(MultiToursB)

# # Cycle time and bounds. Update the MultiTours list with only feasible tours (Possibility to mix basic and random tours)
# # Tmin, Tmax, T_EOQ, T, MultiTours = MT.get_cycle_time(Time, Customers, MultiToursR + MultiToursB, SingleToursR + SingleToursB, n_MultiToursR + n_MultiToursB, vehicleCapacity, delta, phi, h, d)
# Tmin, Tmax, T_EOQ, T, MultiTours = MT.get_cycle_time(Time, Customers, MultiToursB,  SingleToursB, n_MultiToursB, vehicleCapacity, delta, phi, h, d)
# n_MultiTours = len(MultiTours) 

# # Binary representation of the MultiTours
# X = MT.Multitours_Binary(MultiTours, ncustomers)

# # Multitour cost
# Cost = MT.Multitour_Cost(n_MultiTours, ncustomers, psi, T, delta, t, X, phi, h, d)

### Big M (Artificial Variable)

In [5]:
# # Artificial variable / multitour
artificial = [0]
for i in range(1, ncustomers + 1):
    artificial.append(i)
artificial.append(0)
artificial

# Multitours
n_MultiTours = 1
MultiTours = [artificial]
# T = [MT.calculate_travel_time(artificial, Time)]
T = [100]
Cost = [1000000] # Big M
X = MT.Multitours_Binary(MultiTours, ncustomers)

# Column Generation

In [6]:
# Steps 1 to 4 of the column generation algorithm
results = []
RC_star = -1
iteration = 0
while RC_star < -1e-10 and iteration < 50:
    # Restrincted Master Problem
    RMP = IRP_LPref.MasterProblem(MultiTours, ncustomers, nvehicles, Cost, X, LPrelaxation=True)
    Min_TC = IRP_LPref.MinTC_LPref(RMP)
    solutionRMP = IRP_LPref.SolutionMP(RMP)
    DualPrices, DualZero = IRP_LPref.DualPricesMP(RMP)

    # Multi-tour Generation Subproblem
    SP = IRP_MGSP.SubProblem(ncustomers, psi[0], delta, phi, h, d, k, t, vehicleCapacity[0], DualPrices, DualZero)
    RC_star = IRP_MGSP.MinRC(SP)
    new_MultiTour, T_new = IRP_MGSP.SolutionSP(SP)

    # Binary representation of the new MultiTour
    new_x = MT.Multitours_Binary([new_MultiTour], ncustomers)

    # Calculate new cost
    new_cost = MT.Multitour_Cost(1, ncustomers, psi, [T_new], delta, t, new_x, phi, h, d)

    # Check if the new MultiTour is already in the MultiTours list
    # TODO: check the simmytry of the MultiTour
    try:
        idx = MultiTours.index(new_MultiTour)
    except ValueError:
        idx = ""

    # Only add the new MultiTour if it is not already in the MultiTours list or if it has a lower cost
    if idx == "" or (new_cost[0] < Cost[idx] and T[idx]-T_new > 1e-10):
        # Add new MultiTour to the MultiTours list
        MultiTours.append(new_MultiTour)
        n_MultiTours += 1

        # Add bianry representation of the new MultiTour to X
        X = np.append(X, new_x, axis=0)

        # Add new cost and cycle time to the Cost and T arrays
        Cost.append(new_cost[0])
        # T = np.append(T, T_new)
        T.append(T_new)

        # reset count of iterations without improvement
        count = 0
    else:
        count +=1
        if count > 2:
            print("No improvement in the last 2 iterations")
            break
    
    print("Iteration", iteration, ":")
    print("Min reduced cost",RC_star)
    print("Generated MultiTour", new_MultiTour)
    print("Cycle time", T_new)
    print("Min total cost RMP",Min_TC)

    results.append([iteration, RC_star, new_MultiTour, T_new, Min_TC])

    iteration += 1

# Step 5: Solve the Master Problem as a MILP
MP = IRP_LPref.MasterProblem(MultiTours, ncustomers, nvehicles, Cost, X, LPrelaxation=False)
solutionMP = IRP_LPref.SolutionMP(MP)
Min_TC = IRP_LPref.MinTC_LPref(MP)
print("")
print("Integer Master Problem")
print("Solution of the Master Problem", solutionMP)
print("Min total cost MP",Min_TC)
print("")
for sol in solutionMP:
    print(MultiTours[sol[0]], T[sol[0]], Cost[sol[0]])

Set parameter Username
Academic license - for non-commercial use only - expires 2024-11-29
Iteration 0 :
Min reduced cost -993700.0
Generated MultiTour [0, 1, 0]
Cycle time 2.0
Min total cost RMP 1000000.0
Iteration 1 :
Min reduced cost -977583.3333333334
Generated MultiTour [0, 2, 0]
Cycle time 6.0
Min total cost RMP 1000000.0
Iteration 2 :
Min reduced cost -966458.3333333334
Generated MultiTour [0, 3, 0]
Cycle time 4.0
Min total cost RMP 1000000.0
Iteration 3 :
Min reduced cost -960308.3333333334
Generated MultiTour [0, 4, 0]
Cycle time 2.0
Min total cost RMP 1000000.0
Iteration 4 :
Min reduced cost -993850.0
Generated MultiTour [0, 5, 0]
Cycle time 2.0
Min total cost RMP 1000000.0
Iteration 5 :
Min reduced cost -988800.0
Generated MultiTour [0, 6, 0]
Cycle time 2.0
Min total cost RMP 1000000.0
Iteration 6 :
Min reduced cost -993850.0
Generated MultiTour [0, 7, 0]
Cycle time 2.0
Min total cost RMP 1000000.0
Iteration 7 :
Min reduced cost -988800.0
Generated MultiTour [0, 8, 0]
Cycle 

In [7]:
# For example 2, with 4 vehicles the LP of the RMP is not integer M=1000000
# [(3, 0.5), (8, 1.0), (15, 0.5), (17, 0.5), (20, 1.0), (21, 0.5)]
solutionRMP

[(3, 0.5), (8, 1.0), (15, 0.5), (17, 0.5), (20, 1.0), (21, 0.5)]

In [8]:
# plot results
# ut.plot_results(results, initialization=" (Basic tours)")